In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import get_raw_data
import get_preprocessing_data
import analysis_method
import statsmodels.regression.linear_model as sm

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

pv_list = ["0102", "1701", "1702", "0111", "0330", "0331", "0332", "0333", "0101", "0107", "0110", "5501"]
#["0102"(가구주와의 관계), "1701"(근로소득 유무), "1702"(총 연간근로소득), "0111"(학력졸업여부), "0330"(업종8차),
# "0331"(업종9차), "0332"(직종8차), "0333"(직종9차), "0101"(성별), "0107"(만나이), "0110"(학력학교), "5501"(혼인상태)]
hv_list = ["0150", "1406", "2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155",
           "2156", "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
           "2189", "2190", "2191"]
income_code_list = ["2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155", "2156",
                    "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
                    "2189", "2190", "2191"]

cpi_data = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls").loc[0, :]
cpi_data_index = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls", dtype="str").loc[14, :].values[1:]
cpi_data = pd.Series(cpi_data.values[1:], dtype="float")
cpi_data.name = "CPI"
cpi_data.index = cpi_data_index

industry_8 = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="8차")
industry_large = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="대분류")

kospi_return = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\kospi_return.xlsx", index_col=0)

data_num = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21"]

cpi_multiple = get_preprocessing_data.get_cpi_multi(cpi_data, data_num)

data_dict = {}
for i in data_num:
    h_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\h_data\\h_" + i + ".pickle"
    p_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\p_data\\p_" + i + ".pickle"

    h_data = pd.read_pickle(h_path)
    p_data = pd.read_pickle(p_path)

    raw_hdata = get_raw_data.get_hv_data(h_data, hv_list, i)
    raw_pdata = get_raw_data.get_pv_data(p_data, pv_list, i)
    
    fin_p_data = get_preprocessing_data.get_fin_personal(raw_pdata, industry_8, industry_large)
    income_data = get_preprocessing_data.get_income_data(fin_p_data.index, raw_hdata)

    fin_x_value = get_preprocessing_data.get_final_xvalue(raw_hdata, fin_p_data)
    fin_y_value = get_preprocessing_data.get_final_yvalue(income_data, cpi_multiple[i])

    finally_preprocessed_data = get_preprocessing_data.get_total_data(fin_y_value, fin_x_value)

    data_dict[i] = finally_preprocessed_data

C:\Users\JeayongYu\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_df = pd.DataFrame()
for i in data_dict.keys():
    data_df = pd.concat([data_df, data_dict[i]])

_______________________________


### log(Y*)의 계산

In [3]:
temp_edu = data_df.groupby(["0110"]).get_group(1)
temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, 22)
x_mat = analysis_method.get_reg_x(temp_edu).iloc[:, :3]
fer_results = analysis_method.fe_regression_sol(temp_edu, False)

poly_x_mat = analysis_method.polynomial_matrix(temp_edu["0107"], 3) .iloc[:, 1:]
poly_x_mat.index = x_mat.index
fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

In [4]:
fer_results[0][:3]

intercept     7.507209
marriage      0.319783
family_num    0.058957
dtype: float64

In [5]:
fitting_result[1]

intercept    -0.741728
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

In [6]:
fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
fitting_paramters

intercept     6.765481
marriage      0.319783
family_num    0.058957
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

In [7]:
real_data = pd.concat([x_mat, poly_x_mat], axis=1)
real_data

,intercept,marriage,family_num,age^1/10^0,age^2/10^1,age^3/10^2
hhid,,,,,,
21.0,1,1.0,3.0,25.0,62.5,156.25
25.0,1,1.0,4.0,37.0,136.9,506.53
32.0,1,1.0,4.0,59.0,348.1,2053.79
33.0,1,1.0,4.0,42.0,176.4,740.88
40.0,1,1.0,5.0,47.0,220.9,1038.23
...,...,...,...,...,...,...
110085.0,1,1.0,4.0,35.0,122.5,428.75
110102.0,1,1.0,2.0,30.0,90.0,270.00
110122.0,1,1.0,4.0,37.0,136.9,506.53


In [8]:
temp_edu["labor income"]

hhid
21.0        1940.968781
25.0        5822.906343
32.0        2911.453171
33.0        3881.937562
40.0        5822.906343
               ...     
110085.0    6140.000000
110102.0    3500.000000
110122.0    6678.000000
110235.0    6560.000000
110250.0    9700.000000
Name: labor income, Length: 23510, dtype: float64

In [9]:
real_data @ fitting_paramters

hhid
21.0        7.829511
25.0        8.196628
32.0        8.425393
33.0        8.299808
40.0        8.437226
              ...   
110085.0    8.149944
110102.0    7.905564
110122.0    8.196628
110235.0    8.221976
110250.0    8.055704
Length: 23510, dtype: float64

_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 한번 추정하여 rid계산하는 경우)

In [10]:
fitting_paramters

intercept     6.765481
marriage      0.319783
family_num    0.058957
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

----------------------------------------
### shock calculation

In [11]:
group_sigma_df = pd.DataFrame()
group_pvalue_df = pd.DataFrame()
group_rid_df = pd.DataFrame()

start_list = [22, 24, 26]
for i in range(data_df["0110"].max()):
    df_y_star = pd.DataFrame()

    temp_edu = data_df.groupby(["0110"]).get_group(i+1)
    temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, start_list[i])
    
    fer_results = analysis_method.fe_regression_sol(temp_edu, False)
    fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

    fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
    
    for j in data_num:
        temp_edu_split = data_dict[j].groupby(["0110"]).get_group(i+1)
        temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])
        
        log_y = np.log(temp_edu_split["labor income"])
        
        x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
        poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
        poly_x_mat.index = x_mat.index
        
        temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)
    
        f_hat = temp_real_data @ fitting_paramters
        
        log_y_star = log_y - f_hat
        df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

    df_y_star.columns = data_num
    
    temp_varrid_list = []
    temp_rid_list = []
    for k in range(len(data_num)-1):
        temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
        temp_rid_list.append(temp_rid)
        
        temp_varrid_list.append(np.nanvar(temp_rid))
        
    var_rid_series = pd.Series(temp_varrid_list)
    
    rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
    group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)
    
    var_rid_x = np.ones([len(var_rid_series), 2]) * 2
    var_rid_x[:, 0] = var_rid_series.index.values + 1
    var_rid_y = var_rid_series.values
    
    sol = sm.OLS(var_rid_y, var_rid_x)
    sol_result = sol.fit()
    
    sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
    pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])
    
    group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
    group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

group_rid_df.columns = ["high_school", "college", "university"]
group_sigma_df.columns = ["high_school", "college", "university"]
group_pvalue_df.columns = ["high_school", "college", "university"]
group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
group_sigma_df

,high_school,college,university
idiosyncratic_temporary_shock,0.174855,0.145682,0.141262
permanent_shock,0.005461,0.010166,0.005379
idiosyncratic_temporary_shock(p-value),0.000000,0.000000,0.000000
permanent_shock(p-value),0.001000,0.000000,0.009000


----------------------------------------
### Beta & Correlation

In [12]:
ksi = group_rid_df
ksi

,high_school,college,university
02,0.011292,-0.035928,-0.017210
03,0.093463,0.068189,0.046215
04,-0.014766,-0.026539,0.134206
05,0.195853,0.314452,0.109214
06,0.054299,-0.018083,0.038344
07,0.071300,0.107663,0.040838
08,0.030607,0.090565,0.078086
09,0.104878,0.110067,0.075320
10,0.040013,0.111453,0.076239
11,0.032881,-0.001108,0.062704


In [13]:
kospi_return

,코스피
Symbol Name,
1,-42.21
2,49.47
3,82.78
4,-49.11
5,37.47
6,-9.54
7,29.19
8,10.51
9,55.99


In [14]:
kospi_return.index = data_num
kospi_ret = kospi_return.copy() / 100

eta_list = []
beta_list = []
pvalue_list = []
for i in ksi.columns:
    temp_eta = (kospi_ret - 0.02) - np.mean(kospi_ret - 0.02) 
    temp_beta_model = sm.OLS(ksi[i], temp_eta.loc[ksi.index])
    temp_beta_results = temp_beta_model.fit()
    print("----------------------------------------------------------")
    print(i)
    print(temp_beta_results.summary())
    
    eta_list.append(temp_eta)
    beta_list.append(temp_beta_results.params[0])
    pvalue_list.append(temp_beta_results.pvalues[0])
    
beta = pd.DataFrame(beta_list, index=["high_school", "college", "university"], columns=["Beta"])
pvalue = pd.DataFrame(pvalue_list, index=["high_school", "college", "university"], columns=["p-value"])
pd.concat([beta, pvalue], axis=1)

----------------------------------------------------------
high_school
                                 OLS Regression Results                                
Dep. Variable:            high_school   R-squared (uncentered):                   0.252
Model:                            OLS   Adj. R-squared (uncentered):              0.212
Method:                 Least Squares   F-statistic:                              6.389
Date:                Mon, 18 May 2020   Prob (F-statistic):                      0.0205
Time:                        10:17:47   Log-Likelihood:                          27.865
No. Observations:                  20   AIC:                                     -53.73
Df Residuals:                      19   BIC:                                     -52.73
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      

,Beta,p-value
high_school,0.111747,0.020508
college,0.093890,0.146159
university,0.011240,0.793105


In [15]:
corr_list = []
for i in [0, 1, 2]:
    corr_list.append(np.corrcoef(temp_eta.loc[ksi.index, :].코스피, ksi[ksi.columns[i]])[0, 1])

pd.DataFrame(corr_list, index=ksi.columns, columns=["correlation coefficent"])

,correlation coefficent
high_school,0.633465
college,0.334903
university,-0.010961


_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 매번 추정하여 rid계산하는 경우)

In [16]:
group_sigma_df = pd.DataFrame()
group_pvalue_df = pd.DataFrame()
group_rid_df = pd.DataFrame()

start_list = [22, 24, 26]
for i in range(data_df["0110"].max()):
    df_y_star = pd.DataFrame()

    for j in data_num:
        temp_edu_split = data_dict[j].groupby(["0110"]).get_group(i+1)
        temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])
        
        fer_results = analysis_method.fe_regression_sol(temp_edu_split, False)
        fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

        fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
        
        log_y = np.log(temp_edu_split["labor income"])
        
        x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
        poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
        poly_x_mat.index = x_mat.index
        
        temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)
    
        f_hat = temp_real_data @ fitting_paramters
        
        log_y_star = log_y - f_hat
        df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

    df_y_star.columns = data_num
    
    temp_varrid_list = []
    temp_rid_list = []
    for k in range(len(data_num)-1):
        temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
        temp_rid_list.append(temp_rid)
        
        temp_varrid_list.append(np.nanvar(temp_rid))
        
    var_rid_series = pd.Series(temp_varrid_list)
    
    rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
    group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)
    
    var_rid_x = np.ones([len(var_rid_series), 2]) * 2
    var_rid_x[:, 0] = var_rid_series.index.values + 1
    var_rid_y = var_rid_series.values
    
    sol = sm.OLS(var_rid_y, var_rid_x)
    sol_result = sol.fit()
    
    sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
    pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])
    
    group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
    group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

group_rid_df.columns = ["high_school", "college", "university"]
group_sigma_df.columns = ["high_school", "college", "university"]
group_pvalue_df.columns = ["high_school", "college", "university"]
group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
group_sigma_df

,high_school,college,university
idiosyncratic_temporary_shock,0.180054,0.145236,0.140928
permanent_shock,0.003804,0.008621,0.004682
idiosyncratic_temporary_shock(p-value),0.000000,0.000000,0.000000
permanent_shock(p-value),0.028000,0.000000,0.021000


----------------------------------------
### Beta & Correlation

In [17]:
ksi = group_rid_df

kospi_return.index = data_num
kospi_ret = kospi_return.copy() / 100

beta_list = []
pvalue_list = []
eta_list = []
for i in ksi.columns:
    temp_eta = (kospi_ret - 0.02) - np.mean(kospi_ret - 0.02) 
    temp_beta_model = sm.OLS(ksi[i], temp_eta.loc[ksi.index])
    temp_beta_results = temp_beta_model.fit()
    print("----------------------------------------------------------")
    print(i)
    print(temp_beta_results.summary())
    
    eta_list.append(temp_eta)
    beta_list.append(temp_beta_results.params[0])
    pvalue_list.append(temp_beta_results.pvalues[0])
    
beta = pd.DataFrame(beta_list, index=["high_school", "college", "university"], columns=["Beta"])
pvalue = pd.DataFrame(pvalue_list, index=["high_school", "college", "university"], columns=["p-value"])
pd.concat([beta, pvalue], axis=1)

----------------------------------------------------------
high_school
                                 OLS Regression Results                                
Dep. Variable:            high_school   R-squared (uncentered):                   0.006
Model:                            OLS   Adj. R-squared (uncentered):             -0.047
Method:                 Least Squares   F-statistic:                             0.1089
Date:                Mon, 18 May 2020   Prob (F-statistic):                       0.745
Time:                        10:17:50   Log-Likelihood:                          40.175
No. Observations:                  20   AIC:                                     -78.35
Df Residuals:                      19   BIC:                                     -77.35
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      

,Beta,p-value
high_school,0.007882,0.745056
college,0.084456,0.019771
university,0.021894,0.351595


In [18]:
corr_list = []
for i in [0, 1, 2]:
    corr_list.append(np.corrcoef(temp_eta["코스피"].loc[ksi.index], ksi[ksi.columns[i]])[0, 1])

pd.DataFrame(corr_list, index=ksi.columns, columns=["correlation coefficent"])

,correlation coefficent
high_school,0.012356
college,0.537695
university,0.273104
